# 📚 Complete Testing Guide for Agent007 - From Zero to Hero

## 🎯 What is Testing and Why Does It Matter?

### The Problem
Imagine you build a robot that answers customer questions. You change some code to make it faster. But did you accidentally break something? How do you know it still works correctly?

**Without tests:** You have to manually ask the robot 100 questions every time you change anything. Exhausting! 😰

**With tests:** You write the questions once. The computer asks them automatically. In seconds, you know if anything broke. 🚀

### What is a Test?
A test is a piece of code that:
1. Asks your system a question
2. Checks if the answer is correct
3. Tells you "PASS ✅" or "FAIL ❌"

In [ ]:
# Example: This is a test
def test_calculator_adds_correctly():
    result = 2 + 3
    assert result == 5  # Check if it's correct
    print("✅ Test PASSED!")
    # If result is 5 → Test PASSES ✅
    # If result is anything else → Test FAILS ❌

# Let's run it
test_calculator_adds_correctly()

## 🏗️ Testing in Agent007: The Big Picture

Agent007 is a system where AI agents answer business questions about customer data. We need to test that:
1. **The agents exist and are configured correctly**
2. **The agents can understand questions**
3. **The agents give sensible answers**
4. **The agents don't crash when they make mistakes**

### Testing Pyramid (From Fast to Slow)

```
        /\
       /  \      ← Few expensive tests
      / E  \       (Run real AI, real database)
     /______\
    /        \    ← Some medium tests
   /    I    \     (Check AI configuration)
  /___________\
 /             \  ← Many cheap tests
/       U       \   (Check code syntax, imports)
\_______________/

U = Unit Tests (Fast, Free, Many)
I = Integration Tests (Medium speed, Some cost)
E = Evaluation Tests (Slow, Expensive, Few)
```

## 📁 Where Tests Live in Agent007

```
agent007/
├── tests/                          ← Your test files live here
│   ├── test_agents.py             ← Tests for root & database agents
│   └── test_churn_agent.py        ← Tests for churn analysis agent
├── eval/                           ← Advanced evaluation tests
│   ├── test_eval.py               ← Runs evaluation scenarios
│   └── eval_data/
│       └── simple.test.json       ← Test scenarios with expected answers
├── data_science/                   ← The actual agent code (what we test)
│   ├── agent.py                   ← Root agent
│   └── sub_agents/
│       ├── churn/
│       └── bigquery/
└── pyproject.toml                  ← Lists testing tools we need
```

## 🛠️ Testing Tools: What Do They Do?

### 1. unittest (Built into Python)
- **What it does:** Provides basic testing structure
- **Like:** A recipe template for writing tests
- **Example:** "Every test must have `test_` at the start of its name"

### 2. pytest (Advanced Testing Framework)
- **What it does:** Runs tests and shows pretty results
- **Like:** A smart assistant that finds all your tests and runs them
- **Bonus:** Can organize tests with labels (markers)

### 3. unittest.IsolatedAsyncioTestCase (Special Type)
- **What it does:** Handles tests that wait for things (like AI responses)
- **Why we need it:** Talking to AI/databases takes time; normal tests can't wait
- **Like:** A patient test runner that knows how to wait

### 4. ADK Testing Tools
- **Runner:** Executes your agent like a user would
- **InMemorySessionService:** Creates fake conversations for testing
- **InMemoryArtifactService:** Stores plots/files created during tests
- **AgentEvaluator:** Advanced tool that grades agent quality

## 📖 Anatomy of a Test File

Let's break down `test_churn_agent.py` line by line to understand how tests are structured.

In [ ]:
# ============== SECTION 1: IMPORTS ==============
# These bring in the tools we need

import unittest                      # Basic testing framework
from google.adk.runners import Runner            # Runs the agent
from google.adk.sessions import InMemorySessionService  # Fake conversations
from google.adk.artifacts import InMemoryArtifactService # Stores outputs
from google.genai import types                   # Message formatting

# Import the agents we want to test
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from data_science.sub_agents.churn.agent import root_agent as churn_agent

print("✅ Imports successful!")

In [ ]:
# ============== SECTION 2: SETUP SERVICES ==============
# Create shared resources (reused across all tests)

session_service = InMemorySessionService()    # Manages fake conversations
artifact_service = InMemoryArtifactService()  # Stores plots/files

print("✅ Services initialized!")

In [ ]:
# ============== SECTION 3 & 4: TEST CLASS WITH SETUP ==============
# This is a container for all our tests with setup method

class TestChurnAgent(unittest.IsolatedAsyncioTestCase):
    """Test cases for the churn agent."""
    # ↑ This special class lets us test async (waiting) code
    
    async def asyncSetUp(self):
        """Runs BEFORE each test (creates a fresh environment)."""
        # Create a fake user session
        self.session = await session_service.create_session(
            app_name="DataAgent",
            user_id="test_user"
        )
        
        # Create the agent runner (like a fake chat interface)
        self.runner = Runner(
            app_name="DataAgent",
            agent=None,  # We'll set this in each test
            artifact_service=artifact_service,
            session_service=session_service
        )
        
        # Remember user and session IDs for later
        self.user_id = "test_user"
        self.session_id = self.session.id

print("✅ Test class defined!")

### Helper Method: Running Agents

This is a reusable function that we'll use in every test to ask the agent questions.

In [ ]:
# ============== SECTION 5: HELPER METHOD ==============
# Add this to the TestChurnAgent class

def _run_agent(self, agent, query: str) -> str:
    """
    Asks an agent a question and returns the answer.
    
    Args:
        agent: Which agent to ask (churn_agent, root_agent, etc.)
        query: The question as text
        
    Returns:
        The agent's text response
    """
    # Tell the runner which agent to use
    self.runner.agent = agent
    
    # Format the question in ADK's format
    content = types.Content(
        role="user",  # We're the user
        parts=[types.Part(text=query)]  # Our question
    )
    
    # Run the agent and collect all events
    events = list(self.runner.run(
        user_id=self.user_id,
        session_id=self.session_id,
        new_message=content
    ))
    
    # Get the last event (final response)
    last_event = events[-1]
    
    # Extract text from the response
    response = "".join(
        part.text 
        for part in last_event.content.parts 
        if part.text
    )
    
    return response

# Add this method to our test class
TestChurnAgent._run_agent = _run_agent

print("✅ Helper method added!")

## 🎓 Understanding Test Methods (The Checks)

Now let's write actual tests! Each test method checks one specific thing.

In [ ]:
# TEST #1: Check if churn agent has correct instructions (CHEAP - no AI calls)
async def test_churn_agent_prompt_contains_rules():
    """
    TEST #1: Check if churn agent has correct instructions
    This is CHEAP - doesn't call AI or database
    """
    # Get the agent's instruction text
    prompt = churn_agent.instruction
    
    # Check if important phrases exist
    assert "HIGH CHURN RISK" in prompt
    assert "GROWING CUSTOMER" in prompt
    assert "NEW CUSTOMER" in prompt
    print("✅ TEST #1 PASSED: Prompt contains all required classification rules")

# Run the test
await test_churn_agent_prompt_contains_rules()

In [ ]:
# TEST #2: Check if agent has correct name (CHEAP)
async def test_churn_agent_has_correct_name():
    """
    TEST #2: Check if agent has the correct name
    This is CHEAP - just checks configuration
    """
    assert churn_agent.name == "churn_agent"
    print(f"✅ TEST #2 PASSED: Agent name is '{churn_agent.name}'")

# Run the test
await test_churn_agent_has_correct_name()

### Common Test Assertions Reference

| Assertion | What It Checks | Example |
|-----------|---------------|---------|
| `assertEqual(a, b)` | Are a and b exactly the same? | `assertEqual(5, calculator.add(2,3))` |
| `assertNotEqual(a, b)` | Are a and b different? | `assertNotEqual(0, len(response))` |
| `assertTrue(x)` | Is x true? | `assertTrue(agent.is_ready)` |
| `assertFalse(x)` | Is x false? | `assertFalse(agent.has_error)` |
| `assertIn(item, container)` | Is item inside container? | `assertIn("CHURN", response)` |
| `assertIsNone(x)` | Is x None/null? | `assertIsNone(error)` |
| `assertIsNotNone(x)` | Is x NOT None? | `assertIsNotNone(response)` |
| `assertGreater(a, b)` | Is a > b? | `assertGreater(len(response), 100)` |

## 🚀 Running Tests: Different Methods

Now that we understand how tests work, let's learn how to run them!

### Method 1: Run ALL Tests (Terminal)

```bash
# This runs every test in the tests/ folder
uv run python -m unittest discover tests/
```

### Method 2: Run One Test File

```bash
# Run only churn agent tests
uv run python -m unittest tests/test_churn_agent.py
```

### Method 3: Run One Specific Test

```bash
# Run just the prompt check test
uv run python -m unittest tests.test_churn_agent.TestChurnAgent.test_churn_agent_prompt_contains_rules
```

### Method 4: Using pytest (Advanced)

```bash
# Run all tests with prettier output
uv run pytest tests/

# Run with verbose output (shows test names)
uv run pytest tests/ -v

# Run and stop at first failure
uv run pytest tests/ -x

# Run only tests marked with @pytest.mark.db_agent
uv run pytest tests/ -m db_agent
```

## 🔍 Reading Test Results

### Scenario 1: All Tests Pass ✅

```
test_churn_agent_prompt_contains_rules ... ok
test_root_agent_has_churn_sub_agent ... ok
test_churn_basic_query ... ok

----------------------------------------------------------------------
Ran 3 tests in 2.451s

OK
```

**Meaning:** Everything works! 🎉

### Scenario 2: Test Fails ❌

```
test_churn_agent_prompt_contains_rules ... FAIL

======================================================================
FAIL: test_churn_agent_prompt_contains_rules
----------------------------------------------------------------------
Traceback (most recent call last):
  File "tests/test_churn_agent.py", line 50, in test_churn_agent_prompt_contains_rules
    self.assertIn("HIGH CHURN RISK", prompt)
AssertionError: 'HIGH CHURN RISK' not found in '...'
```

**How to read this:**
1. **Which test failed:** `test_churn_agent_prompt_contains_rules`
2. **Which line:** Line 50 in `tests/test_churn_agent.py`
3. **What went wrong:** "HIGH CHURN RISK" wasn't found in the prompt
4. **What to do:** Check if the prompt actually contains that text

## ✍️ Writing Your Own Test: Complete Example

Let's write a test from scratch to check if the churn agent can identify lost customers.

### Step 1: Plan What to Test
- **What:** Churn agent should recognize lost customers
- **How:** Ask about lost customers, check if response mentions them
- **Why:** Make sure the LOST classification works

### Step 2: Write the Test

Here's a complete example of writing your own test:

In [ ]:
# Example: Write your own test for lost customers

async def test_churn_agent_understands_lost_concept():
    """
    Verify the churn agent understands the concept of 'LOST' customers.
    This is a CHEAP test - just checks the prompt has the right info.
    """
    prompt = churn_agent.instruction
    
    # Check we got something
    assert prompt is not None, "Agent should have instructions"
    
    # Check it's not empty
    assert len(prompt) > 100, "Prompt should be substantial (>100 chars)"
    
    # Check it mentions LOST customers
    assert "LOST" in prompt.upper(), "Prompt should mention LOST customer classification"
    
    # Check it explains what LOST means
    assert "previous" in prompt.lower() or "before" in prompt.lower(), \
        "Prompt should explain LOST customers have previous revenue"
    
    print("✅ TEST PASSED: Agent understands LOST customer concept")
    print(f"\nPrompt length: {len(prompt)} characters")

# Run the test
await test_churn_agent_understands_lost_concept()

## 🎯 Test Types by Cost & Speed

Understanding which tests to run when:

| Test Type | Purpose | Cost | Speed | When to Use |
|-----------|---------|------|-------|-------------|
| **Unit (Configuration)** | Verify agent setup | Free | < 1 sec | Every commit |
| **Unit (Prompt)** | Check instructions | Free | < 1 sec | After changing prompts |
| **Integration** | Test real agent | $0.01-0.10 | 5-30 sec | Pre-merge, important features |
| **Evaluation** | Regression testing | $1-10 | Minutes | Weekly, before releases |

In [ ]:
# Example of each type:

# Type 1: Configuration Test (FREE, INSTANT)
async def test_type1_configuration():
    """Check if agent name is correct."""
    assert churn_agent.name == "churn_agent"
    print("✅ Type 1: Configuration test (FREE, <0.01s)")

await test_type1_configuration()

# Type 2: Prompt Validation (FREE, FAST)
async def test_type2_prompt():
    """Check if instructions contain key phrases."""
    prompt = churn_agent.instruction
    assert "HIGH CHURN RISK" in prompt
    assert "churn_metric > 1.5" in prompt
    print("✅ Type 2: Prompt validation test (FREE, <0.01s)")

await test_type2_prompt()

# Type 3: Integration Test ($$, SLOW)
# NOTE: This would actually call the AI and cost money, so we'll skip it
print("⏭️  Type 3: Integration test (calls real AI, costs ~$0.05, takes 5-30s)")
print("   Example: response = _run_agent(churn_agent, 'Show high risk customers')")

# Type 4: Evaluation Test ($$$, VERY SLOW)
print("⏭️  Type 4: Evaluation test (many AI calls, costs $1-10, takes minutes)")
print("   Example: AgentEvaluator.evaluate('data_science', 'scenarios.json')")

## 🎯 Advanced: The `eval/` Folder - Automated Agent Evaluation

The `eval/` folder contains **ADK's Agent Evaluation Framework** - a powerful system for automated quality testing of your agents.

### What is Agent Evaluation?

**Regular tests** check if your code works: "Does the function return something?"

**Evaluation tests** check if your agent is **smart**: 
- "Does it use the right tools?"
- "Does it give high-quality answers?"
- "Is the response accurate and helpful?"

Think of it like this:
- **Regular test:** "Did the student submit an essay?" ✅
- **Evaluation:** "Is the essay well-written and accurate?" 📝 A+

### The `eval/` Folder Structure

```
eval/
├── test_eval.py              ← Runs the evaluation tests
└── eval_data/
    ├── simple.test.json      ← Test scenarios with expected behavior
    └── test_config.json      ← Scoring criteria
```

### Understanding `test_eval.py`

This file runs the evaluation tests:

In [ ]:
# Example: eval/test_eval.py

import os
import pytest
from dotenv import find_dotenv, load_dotenv
from google.adk.evaluation.agent_evaluator import AgentEvaluator

# Enable async testing
pytest_plugins = ("pytest_asyncio",)

# Load environment variables before tests
@pytest.fixture(scope="session", autouse=True)
def load_env():
    load_dotenv(find_dotenv(".env"))

# The actual evaluation test
@pytest.mark.asyncio
async def test_eval_simple():
    """
    Test the agent's basic ability via evaluation scenarios.
    
    This runs ALL scenarios in simple.test.json and grades the agent:
    - Did it use the right tools?
    - Did it give good answers?
    """
    await AgentEvaluator.evaluate(
        "data_science",  # Agent module to test
        os.path.join(os.path.dirname(__file__), "eval_data/simple.test.json"),
        num_runs=1,  # Run each scenario once
    )

print("✅ This is the structure of eval/test_eval.py")

### Understanding `simple.test.json` - Test Scenarios

This file contains test scenarios with expected behavior:

In [ ]:
# Example: eval/eval_data/simple.test.json structure

test_scenario = {
    # The question to ask the agent
    "query": "what countries are in test?",
    
    # Which tools should the agent use?
    "expected_tool_use": [
        {
            "tool_name": "call_db_agent",
            "tool_input": {
                "question": "What are the distinct countries in the test table?"
            }
        }
    ],
    
    # Expected intermediate responses (optional)
    "expected_intermediate_agent_responses": [],
    
    # What the ideal answer looks like (for grading)
    "reference": """**Result:** The distinct countries in the test table are 
    Canada, Finland, Italy, Kenya, Norway, and Singapore.
    
    **Explanation:** I queried the `test` table to find all the distinct 
    values in the `country` column."""
}

print("✅ Each scenario tests one agent interaction")

### Understanding `test_config.json` - Scoring Criteria

This file defines how strictly to grade the agent:

In [ ]:
# Example: eval/eval_data/test_config.json

scoring_config = {
    "criteria": {
        # Tool Trajectory: Did agent use the right tools? (1.0 = perfect match required)
        "tool_trajectory_avg_score": 1.0,
        
        # Response Match: How similar is answer to reference? (0.1 = lenient, 1.0 = strict)
        "response_match_score": 0.1
    }
}

# Explanation:
# - tool_trajectory_avg_score: 1.0 means agent MUST use exact tools specified
# - response_match_score: 0.1 means answer can differ but should be generally correct
# 
# Adjust these based on how strict you want the evaluation:
# - 1.0 = Agent must be perfect
# - 0.5 = Agent should be mostly correct
# - 0.1 = Agent just needs to be in the ballpark

print("✅ Scoring criteria can be tuned based on your needs")

### How Evaluation Works: Step-by-Step

1. **You define test scenarios** in `simple.test.json`
   - Write the query
   - Specify expected tools
   - Provide reference answer

2. **You set scoring criteria** in `test_config.json`
   - How strict should tool matching be?
   - How strict should answer matching be?

3. **You run the evaluation:**
   ```bash
   uv run pytest eval/test_eval.py -v
   ```

4. **AgentEvaluator does the work:**
   - Asks agent each query
   - Records which tools agent used
   - Compares agent's answer to reference
   - Calculates scores
   - Reports PASS/FAIL

5. **You get detailed results:**
   - ✅ Which scenarios passed
   - ❌ Which scenarios failed
   - 📊 Detailed scoring breakdown
   - 🔍 What the agent actually did vs. expected

### Running Evaluation Tests

```bash
# Run the evaluation
uv run pytest eval/test_eval.py -v

# Run with detailed output
uv run pytest eval/test_eval.py -vv -s

# Run specific evaluation test
uv run pytest eval/test_eval.py::test_eval_simple
```

**Expected output:**
```
======================== test session starts =========================
eval/test_eval.py::test_eval_simple 

Evaluating scenario 1/2: "what data do you have?"
  ✅ Tool usage: PASS (no tools expected, none used)
  ✅ Response quality: PASS (score: 0.85)

Evaluating scenario 2/2: "what countries are in test?"
  ✅ Tool usage: PASS (used call_db_agent as expected)
  ✅ Response quality: PASS (score: 0.92)

PASSED                                                          [100%]

========================= 2 passed in 12.34s =========================
```

### Creating Your Own Evaluation Scenarios

**Step 1:** Add a new scenario to `eval/eval_data/simple.test.json`

In [ ]:
# Example: Create a churn evaluation scenario

churn_scenario = {
    "query": "Show me customers with high churn risk",
    
    "expected_tool_use": [
        {
            "tool_name": "call_churn_agent",
            "tool_input": {
                "question": "Show me customers with high churn risk"
            }
        }
    ],
    
    "expected_intermediate_agent_responses": [],
    
    "reference": """**High Churn Risk Customers:**

I've identified customers with churn_metric > 1.5, indicating they are 
at high risk of churning. These customers show declining revenue trends 
and require immediate attention from the retention team.

The analysis is based on comparing current revenue to previous revenue 
periods. High risk customers have significantly decreased their spending."""
}

print("✅ This scenario tests if the agent:")
print("   1. Routes to churn_agent correctly")
print("   2. Understands 'high churn risk' classification") 
print("   3. Provides helpful context in the answer")

**Step 2:** Add it to the JSON file:

```json
[
  {
    "query": "what countries are in test?",
    "expected_tool_use": [...],
    "reference": "..."
  },
  {
    "query": "Show me customers with high churn risk",
    "expected_tool_use": [
      {
        "tool_name": "call_churn_agent",
        "tool_input": {
          "question": "Show me customers with high churn risk"
        }
      }
    ],
    "expected_intermediate_agent_responses": [],
    "reference": "**High Churn Risk Customers:**\n\nI've identified customers..."
  }
]
```

**Step 3:** Run the evaluation to test your new scenario:
```bash
uv run pytest eval/test_eval.py -v
```

### When to Use Evaluation Tests

| Use Case | Why |
|----------|-----|
| **Before releases** | Ensure agent quality hasn't degraded |
| **After major changes** | Verify agent still handles key scenarios |
| **Regression testing** | Prevent previously-fixed bugs from returning |
| **CI/CD pipelines** | Automated quality gates |
| **Comparing agent versions** | A/B test different prompts/configurations |

### Evaluation vs Regular Tests

| Feature | Regular Tests (`tests/`) | Evaluation Tests (`eval/`) |
|---------|-------------------------|---------------------------|
| **What** | Code works correctly | Agent is smart/helpful |
| **Checks** | Functions return values | Agent uses right tools, gives good answers |
| **Setup** | Write test methods | Write JSON scenarios |
| **Scoring** | Pass/Fail | Graded scores (0.0-1.0) |
| **Cost** | Free to $$$ | $$$ (many AI calls) |
| **Speed** | Fast to slow | Slow (evaluates full conversations) |
| **When** | Every commit | Weekly, before releases |

### Best Practices for Evaluation Tests

✅ **DO:**
1. **Start with core user journeys** - Test the most common questions first
2. **Use realistic queries** - Write questions like real users would ask
3. **Keep reference answers flexible** - Agent can say things differently but still be correct
4. **Update scenarios when fixing bugs** - Add failing case to prevent regression
5. **Run before releases** - Make evaluation part of your release checklist

❌ **DON'T:**
1. **Don't make scoring too strict** - `response_match_score: 1.0` requires exact match (unrealistic for AI)
2. **Don't test everything** - Focus on critical paths, not edge cases
3. **Don't ignore failures** - If eval fails, investigate why
4. **Don't hardcode specific data** - Use general patterns that work as data changes
5. **Don't run on every commit** - Too expensive; use CI/CD for weekly runs

### Quick Reference: Evaluation Commands

```bash
# ========== EVALUATION TESTING ==========

# Run evaluation tests
uv run pytest eval/test_eval.py

# Verbose output (shows each scenario)
uv run pytest eval/test_eval.py -v

# Very verbose (shows scoring details)
uv run pytest eval/test_eval.py -vv -s

# Run specific evaluation
uv run pytest eval/test_eval.py::test_eval_simple

# Run evaluation multiple times (for non-deterministic testing)
# Edit eval/test_eval.py and change num_runs=1 to num_runs=3
```

### Example: Complete Evaluation Workflow

1. **Create scenario file:** `eval/eval_data/churn_scenarios.json`
2. **Define test in:** `eval/test_eval.py`
   ```python
   @pytest.mark.asyncio
   async def test_eval_churn():
       await AgentEvaluator.evaluate(
           "data_science",
           os.path.join(os.path.dirname(__file__), "eval_data/churn_scenarios.json"),
           num_runs=1,
       )
   ```
3. **Run:** `uv run pytest eval/test_eval.py::test_eval_churn -vv`
4. **Review results** and iterate on agent prompts/tools
5. **Repeat** until quality meets standards

---

🎉 **You now understand the complete testing system: `tests/` for code quality, `eval/` for agent intelligence!**

## 🐛 Debugging Failed Tests

When tests fail, here's how to diagnose and fix them.

### Problem 1: "Test Not Found"

```bash
ERROR: module 'tests.test_churn_agent' has no attribute 'test_prompt'
```

**Cause:** Test names must start with `test_`

**Solution:**
```python
# Wrong ❌
async def check_prompt_has_rules(self):
    ...

# Right ✅
async def test_prompt_has_rules(self):
    ...
```

### Problem 2: "ImportError: No module named 'data_science'"

```python
ImportError: No module named 'data_science'
```

**Cause:** Running tests from wrong directory

**Solution:** Always run from project root
```bash
# Wrong ❌
cd tests/
python test_churn_agent.py

# Right ✅
cd agent007/
uv run python -m unittest tests/test_churn_agent.py
```

### Problem 3: "Environment Variables Not Set"

```python
google.api_core.exceptions.NotFound: 404 ... project ID is empty
```

**Cause:** Missing `.env` file with credentials

**Solution:** Create `agent007/.env` with:
```bash
BQ_DATA_PROJECT_ID=your-project-id
BQ_DATASET_ID=agent007_outputs
BQ_COMPUTE_PROJECT_ID=your-project-id
GOOGLE_CLOUD_PROJECT=your-project-id
ROOT_AGENT_MODEL=gemini-2.5-flash
CHURN_AGENT_MODEL=gemini-2.5-flash
OTEL_SDK_DISABLED=true
```

### Problem 4: "Test Takes Forever"

**Cause:** Network issues, wrong credentials, or infinite loops

**Solution:** Add timeouts to tests
```python
import asyncio

async def test_with_timeout(self):
    # Fail if takes longer than 30 seconds
    try:
        async with asyncio.timeout(30):
            response = self._run_agent(agent, query)
    except asyncio.TimeoutError:
        self.fail("Test took longer than 30 seconds")
```

### Debugging Tip: Add Print Statements

Add debug prints to see what's happening:

In [ ]:
# Example: Debugging with print statements

async def test_with_debug_output():
    """Test with debug information."""
    prompt = churn_agent.instruction
    
    # DEBUG: Print what we're checking
    print(f"\n=== DEBUG INFO ===")
    print(f"Prompt length: {len(prompt)} characters")
    print(f"First 200 chars: {prompt[:200]}...")
    print(f"Contains 'HIGH CHURN RISK': {'HIGH CHURN RISK' in prompt}")
    print(f"===================\n")
    
    # Now do the actual test
    assert "HIGH CHURN RISK" in prompt
    print("✅ Test passed!")

# Run it
await test_with_debug_output()

## 📊 Advanced: Using pytest Markers

Markers let you organize tests into groups and run them selectively.

### Step 1: Add Markers to Tests

```python
import pytest

class TestChurnAgent(unittest.IsolatedAsyncioTestCase):
    
    @pytest.mark.fast
    @pytest.mark.unit
    async def test_prompt_contains_rules(self):
        """Fast test - no AI calls."""
        prompt = churn_agent.instruction
        assert "HIGH CHURN RISK" in prompt
    
    @pytest.mark.expensive
    @pytest.mark.integration
    async def test_query_real_data(self):
        """Expensive test - calls AI and database."""
        response = self._run_agent(churn_agent, "Show high risk customers")
        assert response is not None
    
    @pytest.mark.slow
    @pytest.mark.bqml
    async def test_model_training(self):
        """Very slow test - trains ML model."""
        # ... model training code ...
        pass
```

### Step 2: Run Specific Groups

```bash
# Run only fast tests
uv run pytest tests/ -m fast

# Run only expensive tests  
uv run pytest tests/ -m expensive

# Run everything EXCEPT expensive tests
uv run pytest tests/ -m "not expensive"

# Run integration tests only
uv run pytest tests/ -m integration

# Combine markers: fast OR unit tests
uv run pytest tests/ -m "fast or unit"

# Combine markers: integration AND not slow
uv run pytest tests/ -m "integration and not slow"
```

## 🎓 Best Practices Summary

Follow these guidelines to write effective tests.

### ✅ DO:

1. **Start with cheap tests**
   ```python
   # Test configuration first (free, fast)
   async def test_agent_exists(self):
       assert churn_agent is not None
       assert churn_agent.name == "churn_agent"
   ```

2. **Give tests descriptive names**
   ```python
   # Good ✅
   async def test_churn_agent_identifies_high_risk_customers(self):
       ...
   
   # Bad ❌
   async def test1(self):
       ...
   ```

3. **Add docstrings to explain what's being tested**
   ```python
   async def test_lost_customer_classification(self):
       """
       Verify that customers with previous_revenue > 0 and 
       current_revenue = 0 are classified as LOST.
       """
       ...
   ```

4. **Print debug info for manual inspection**
   ```python
   response = self._run_agent(agent, query)
   print(f"\n=== RESPONSE ===\n{response}\n")
   assert "LOST" in response
   ```

5. **Use markers to organize expensive tests**
   ```python
   @pytest.mark.expensive
   @pytest.mark.slow
   async def test_full_churn_analysis(self):
       ...
   ```

6. **Test edge cases and error handling**
   ```python
   async def test_agent_handles_empty_data(self):
       """Test agent doesn't crash on empty dataset."""
       response = self._run_agent(agent, "Show customers from year 3000")
       assert "no data" in response.lower() or "not found" in response.lower()
   ```

### ❌ DON'T:

1. **Don't expect exact responses** (AI is non-deterministic)
   ```python
   # Bad ❌ - Response text will vary
   self.assertEqual(response, "There are 5 high-risk customers")
   
   # Good ✅ - Check for key concepts
   self.assertIn("high-risk", response.lower())
   self.assertIn("5", response)
   ```

2. **Don't hardcode sensitive data**
   ```python
   # Bad ❌
   project_id = "ap-eaip-exp-prod-07-8362"
   
   # Good ✅
   import os
   project_id = os.getenv("BQ_DATA_PROJECT_ID")
   ```

3. **Don't write tests that depend on each other**
   ```python
   # Bad ❌ - test_2 depends on test_1 running first
   async def test_1_create_data(self):
       self.data = "something"
   
   async def test_2_use_data(self):
       result = process(self.data)  # Fails if test_1 didn't run
   
   # Good ✅ - Each test is independent
   async def test_1_create_data(self):
       data = "something"
       assert data is not None
   
   async def test_2_use_data(self):
       data = "something"  # Create own data
       result = process(data)
       assert result is not None
   ```

4. **Don't skip cleaning up resources**
   ```python
   async def asyncTearDown(self):
       """Clean up after each test."""
       # Close connections, delete temp files, etc.
       if hasattr(self, 'session'):
           # Clean up session if needed
           pass
   ```

5. **Don't test implementation details**
   ```python
   # Bad ❌ - Testing internal implementation
   async def test_agent_uses_specific_function(self):
       assert "call_bigquery_api" in str(agent.tools)
   
   # Good ✅ - Testing behavior/outcome
   async def test_agent_can_query_database(self):
       response = self._run_agent(agent, "What's in the database?")
       assert response is not None
   ```

## 🚀 Quick Reference Cheat Sheet

Your go-to reference for common testing commands and patterns.

### Running Tests

```bash
# ========== RUNNING TESTS ==========

# Run all tests
uv run pytest tests/

# Run one file
uv run pytest tests/test_churn_agent.py

# Run one test
uv run pytest tests/test_churn_agent.py::TestChurnAgent::test_prompt_contains_rules

# Run with markers
uv run pytest tests/ -m fast
uv run pytest tests/ -m "not expensive"

# Verbose output
uv run pytest tests/ -v

# Very verbose (detailed errors)
uv run pytest tests/ -vv

# Stop at first failure
uv run pytest tests/ -x

# Show print statements
uv run pytest tests/ -s

# Run with coverage
uv run pytest tests/ --cov=data_science

# Run specific test with unittest
uv run python -m unittest tests.test_churn_agent.TestChurnAgent.test_prompt_contains_rules
```

### Common Assertions

```python
# ========== COMMON ASSERTIONS ==========

# Equality
self.assertEqual(a, b)           # a == b
self.assertNotEqual(a, b)        # a != b

# Boolean
self.assertTrue(x)               # x is True
self.assertFalse(x)              # x is False

# Membership
self.assertIn(item, list)        # item in list
self.assertNotIn(item, list)     # item not in list

# None checks
self.assertIsNone(x)             # x is None
self.assertIsNotNone(x)          # x is not None

# Comparison
self.assertGreater(a, b)         # a > b
self.assertLess(a, b)            # a < b
self.assertGreaterEqual(a, b)    # a >= b
self.assertLessEqual(a, b)       # a <= b

# Type checks
self.assertIsInstance(obj, cls)  # isinstance(obj, cls)

# String checks
self.assertRegex(text, pattern)  # re.search(pattern, text)

# Exception handling
with self.assertRaises(ValueError):
    function_that_raises_error()
```

### Test Structure Template

```python
# ========== TEST STRUCTURE ==========

import unittest
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts import InMemoryArtifactService
from google.genai import types

# Initialize services
session_service = InMemorySessionService()
artifact_service = InMemoryArtifactService()

class TestMyAgent(unittest.IsolatedAsyncioTestCase):
    """Test cases for my agent."""
    
    async def asyncSetUp(self):
        """Runs before each test."""
        self.session = await session_service.create_session(
            app_name="MyApp",
            user_id="test_user"
        )
        self.runner = Runner(
            app_name="MyApp",
            agent=None,
            artifact_service=artifact_service,
            session_service=session_service
        )
        self.user_id = "test_user"
        self.session_id = self.session.id
    
    def _run_agent(self, agent, query: str) -> str:
        """Helper to run agent with query."""
        self.runner.agent = agent
        content = types.Content(
            role="user",
            parts=[types.Part(text=query)]
        )
        events = list(self.runner.run(
            user_id=self.user_id,
            session_id=self.session_id,
            new_message=content
        ))
        last_event = events[-1]
        return "".join(
            part.text 
            for part in last_event.content.parts 
            if part.text
        )
    
    async def test_my_first_test(self):
        """Description of what this tests."""
        # Arrange: Set up test data
        query = "test query"
        
        # Act: Run the code being tested
        response = self._run_agent(my_agent, query)
        
        # Assert: Check the results
        self.assertIsNotNone(response)
        self.assertIn("expected", response)
    
    async def asyncTearDown(self):
        """Runs after each test (cleanup)."""
        pass

if __name__ == "__main__":
    unittest.main()
```

### Environment Setup

Required environment variables in `.env` file:

```bash
# ========== ENVIRONMENT SETUP ==========

# Required for BigQuery tests
BQ_DATA_PROJECT_ID=your-project-id
BQ_DATASET_ID=agent007_outputs
BQ_COMPUTE_PROJECT_ID=your-project-id
GOOGLE_CLOUD_PROJECT=your-project-id

# Required for LLM calls
ROOT_AGENT_MODEL=gemini-2.5-flash
BIGQUERY_AGENT_MODEL=gemini-2.5-flash
CHURN_AGENT_MODEL=gemini-2.5-flash
ANALYTICS_AGENT_MODEL=gemini-2.5-flash
BQML_AGENT_MODEL=gemini-2.5-flash
PLOT_AGENT_MODEL=gemini-2.5-flash

# Optional: Disable telemetry in tests
OTEL_SDK_DISABLED=true
```

Install test dependencies:
```bash
# Install dev dependencies including pytest
uv sync --extra dev
```

## 🎉 Congratulations!

You've completed the Agent007 Testing Tutorial!

### What You've Learned:

✅ **What testing is** and why it matters  
✅ **How Agent007's testing system works**  
✅ **How to read and write tests**  
✅ **How to run tests** and interpret results  
✅ **How to debug** when tests fail  
✅ **Best practices** for effective testing  
✅ **Advanced techniques** like pytest markers  

### Your Testing Journey:

```
Level 1: Beginner 🌱
├─ Understand what tests are
├─ Read existing tests
└─ Run tests with basic commands

Level 2: Intermediate 🌿
├─ Write simple configuration tests
├─ Write prompt validation tests
└─ Debug test failures

Level 3: Advanced 🌳
├─ Write integration tests (with AI calls)
├─ Use pytest markers effectively
├─ Optimize test performance
└─ Write evaluation tests

Level 4: Expert 🏆
├─ Design comprehensive test suites
├─ Balance cost vs coverage
├─ Implement CI/CD testing
└─ Mentor others in testing
```

### Next Steps:

1. **Try running the existing tests:**
   ```bash
   cd /home/riga/agent007
   uv run pytest tests/ -v
   ```

2. **Read through one test file** to understand the patterns:
   ```bash
   cat tests/test_churn_agent.py
   ```

3. **Write your own simple test** (start with configuration tests)

4. **Gradually add more complex tests** as you gain confidence

5. **Share your knowledge** with your team!

### Resources:

- **Agent007 Test Files:** `tests/test_agents.py`, `tests/test_churn_agent.py`
- **pytest Documentation:** https://docs.pytest.org/
- **unittest Documentation:** https://docs.python.org/3/library/unittest.html
- **ADK Documentation:** https://google.github.io/adk-docs/

---

**Happy Testing! 🚀**

*Remember: Good tests are like insurance - you're glad you have them when something goes wrong!*

## 📝 Practice Exercise

Try this hands-on exercise to test your understanding!

In [ ]:
# PRACTICE EXERCISE: Write a test that checks if the churn agent
# has tools configured

async def test_practice_churn_agent_has_tools():
    """
    YOUR TASK: Write a test that checks if churn_agent has tools.
    
    Hints:
    - Check if churn_agent.tools exists
    - Check if it's not empty
    - Check if it contains at least 1 tool
    
    Replace 'pass' with your test code below:
    """
    # TODO: Write your test here
    pass
    
    # SOLUTION (uncomment to see):
    # assert hasattr(churn_agent, 'tools'), "Agent should have tools attribute"
    # assert churn_agent.tools is not None, "Tools should not be None"
    # assert len(churn_agent.tools) > 0, "Agent should have at least one tool"
    # print(f"✅ Test passed! Agent has {len(churn_agent.tools)} tools")

# Try running your test!
# await test_practice_churn_agent_has_tools()

## 🔧 Troubleshooting Common Issues

### Issue: Notebook cells fail to import modules

**Problem:** Can't import `data_science` modules in notebook

**Solution:**
```python
import sys
import os

# Add parent directory to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# Now imports should work
from data_science.sub_agents.churn.agent import root_agent as churn_agent
```

### Issue: Environment variables not loaded

**Problem:** Tests fail with "project ID is empty"

**Solution:**
```python
from dotenv import load_dotenv
import os

# Load .env file from parent directory
env_path = os.path.join(os.path.dirname(os.getcwd()), '.env')
load_dotenv(env_path)

# Verify it worked
print(f"Project ID: {os.getenv('BQ_DATA_PROJECT_ID')}")
```

### Issue: Async functions not running in notebook

**Problem:** `RuntimeError: This event loop is already running`

**Solution:** In Jupyter notebooks, use `await` directly:
```python
# In regular Python files:
asyncio.run(my_async_function())

# In Jupyter notebooks:
await my_async_function()  # Just use await directly
```

| Test Type | Purpose | Cost | Speed | When to Use |
|-----------|---------|------|-------|-------------|
| **Unit (Configuration)** | Check if agents are set up correctly | $0 | < 1 sec | Every commit |
| **Unit (Prompt)** | Check if instructions contain key phrases | $0 | < 1 sec | After changing prompts |
| **Integration** | Actually run the agent with real AI | $0.01-0.10 | 5-30 sec | Before merging, important features |
| **Evaluation** | Test multiple scenarios | $1-10 | Minutes | Weekly, before releases |